<a href="https://colab.research.google.com/github/kdChen07/progress-and-problems/blob/main/driving_cost_calculation_OSRM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#使用OSRM（开源路由机器）计算驾驶距离，并将输出作为csv文件

##--将用于距离矩阵计算的数据子集--

In [1]:
!pip install access
!pip install logging
!pip install geopandas

     |████████████████████████████████| 96 kB 587 kB/s 
ERROR: Could not find a version that satisfies the requirement logging (from versions: 0.4.9.6)
ERROR: No matching distribution found for logging
     |████████████████████████████████| 994 kB 425 kB/s 
     |████████████████████████████████| 15.4 MB 35 kB/s 
     |████████████████████████████████| 6.6 MB 5.5 MB/s 


In [2]:
from access import Access, weights, datasets
import logging
import pandas as pd
import geopandas as gpd

import requests
import json

## Get the centroid of MSOA (demand points)

In [3]:
MSOA_points=gpd.read_file('https://github.com/LingruFeng/dissertation/blob/main/data_github/England_basemap/MSOA_Population_Weighted_Centroids_with_population.gpkg?raw=true')
#MSOA_points = MSOA_points.to_crs('epsg:27700')
MSOA_points = MSOA_points.to_crs('epsg:4326')
MSOA_points['MSOA_lon'] = MSOA_points['geometry'].x
MSOA_points['MSOA_lat'] = MSOA_points['geometry'].y

In [4]:
MSOA_points

,objectid,msoa11cd,pop0-49,pop50-59,pop60-69,pop70-79,pop80-89,pop90+,total_pop,geometry,MSOA_lon,MSOA_lat
0,1,E02002536,5688,1416,1132,891,540,104,9771,POINT (-1.29577 54.61069),-1.295773,54.610691
1,2,E02002537,5578,1251,836,545,386,45,8641,POINT (-1.27726 54.61131),-1.277263,54.611313
2,3,E02002534,2547,824,863,967,407,85,5693,POINT (-1.05346 54.52765),-1.053458,54.527646
3,4,E02002535,4959,1636,1385,870,416,70,9336,POINT (-1.28729 54.62215),-1.287295,54.622148
4,5,E02002532,3783,970,810,655,342,95,6655,POINT (-1.05793 54.53718),-1.057931,54.537176
...,...,...,...,...,...,...,...,...,...,...,...,...
6786,6787,E02004669,3697,1171,1156,946,433,93,7496,POINT (-2.09642 51.94596),-2.096424,51.945960
6787,6788,E02006096,3893,1086,805,804,344,69,7001,POINT (-2.95612 50.87855),-2.956122,50.878552
6788,6789,E02003088,4007,1257,1230,1194,577,179,8444,POINT (-2.81548 51.32678),-2.815479,51.326784
6789,6790,E02006070,5161,1163,906,670,316,75,8291,POINT (-2.97724 51.12711),-2.977239,51.127113


In [5]:
MSOA_points_df = MSOA_points[['msoa11cd','total_pop','MSOA_lon','MSOA_lat']]
MSOA_points_df

,msoa11cd,total_pop,MSOA_lon,MSOA_lat
0,E02002536,9771,-1.295773,54.610691
1,E02002537,8641,-1.277263,54.611313
2,E02002534,5693,-1.053458,54.527646
3,E02002535,9336,-1.287295,54.622148
4,E02002532,6655,-1.057931,54.537176
...,...,...,...,...
6786,E02004669,7496,-2.096424,51.945960
6787,E02006096,7001,-2.956122,50.878552
6788,E02003088,8444,-2.815479,51.326784
6789,E02006070,8291,-2.977239,51.127113


## read vaccination site point data (supply points)

In [6]:
site = gpd.read_file('https://github.com/LingruFeng/dissertation/blob/main/data_github/site_2_26_clean/vaccination_site.gpkg?raw=true')
# site = site.to_crs('epsg:27700')
site = site.to_crs('epsg:4326')
site['site_lon'] = site['geometry'].x
site['site_lat'] = site['geometry'].y

In [7]:
site_df = site[['index','site_lon','site_lat']]
site_df

,index,site_lon,site_lat
0,0,-0.471211,52.128622
1,1,-0.474637,51.894793
2,2,-0.093771,51.468063
3,3,-0.093771,51.468063
4,4,-0.484367,51.891807
...,...,...,...
1595,1595,-4.865818,50.515949
1596,1596,-0.205835,51.433441
1597,1597,-0.173830,51.514861
1598,1598,-1.879170,50.716914


In [8]:
site

,index,supply_value,geometry,site_lon,site_lat
0,0,1,POINT (-0.47121 52.12862),-0.471211,52.128622
1,1,1,POINT (-0.47464 51.89479),-0.474637,51.894793
2,2,1,POINT (-0.09377 51.46806),-0.093771,51.468063
3,3,1,POINT (-0.09377 51.46806),-0.093771,51.468063
4,4,1,POINT (-0.48437 51.89181),-0.484367,51.891807
...,...,...,...,...,...
1595,1595,1,POINT (-4.86582 50.51595),-4.865818,50.515949
1596,1596,1,POINT (-0.20584 51.43344),-0.205835,51.433441
1597,1597,1,POINT (-0.17383 51.51486),-0.173830,51.514861
1598,1598,1,POINT (-1.87917 50.71691),-1.879170,50.716914


# Calculate the Euclidean distance from demand to supply locations 
## --threshold: 10 miles (16093.44 meters)--

In [9]:
# Load each of the example datasets which correspond to the demand (population) and supply ('hospital','pcn','pharmacy','vc') respectively.
distance = Access(demand_df = MSOA_points,
           demand_index='msoa11cd',
           demand_value='total_pop',
           supply_df= site,
           supply_index= 'index',
           supply_value=['supply_value']
                 )
# reproject
distance.demand_df = distance.demand_df.to_crs('epsg:27700')
distance.supply_df = distance.supply_df.to_crs('epsg:27700')

In [10]:
##### Take 48280.32 meters (30 miles) as the calculation threshold
distance.create_euclidean_distance(threshold = 16093.44)
distance.cost_df

,euclidean,origin,dest
0,4573.215379,E02002536,122
1,8449.406849,E02002536,127
2,8217.024557,E02002536,137
3,9944.296655,E02002536,836
4,6707.554555,E02002536,838
...,...,...,...
555153,12599.536095,E02003088,1204
555154,5650.422116,E02006070,1278
555155,14527.210489,E02006679,1176
555156,9895.393238,E02006679,1177


In [11]:
euclidean_distance_matrix = distance.cost_df

In [ ]:
euclidean_distance_matrix = pd.merge(left = euclidean_distance_matrix, 
                                     right = site_df, 
                                     how = 'left', 
                                     left_on = 'dest', 
                                     right_on = 'index')

euclidean_distance_matrix = pd.merge(left = euclidean_distance_matrix, 
                                     right = MSOA_points_df, 
                                     how = 'left', 
                                     left_on = 'origin', 
                                     right_on = 'msoa11cd')

In [ ]:
euclidean_distance_matrix.drop(columns=['msoa11cd','index','total_pop'], axis=1,inplace=True)
euclidean_distance_matrix

,euclidean,origin,dest,site_lon,site_lat,MSOA_lon,MSOA_lat
0,4573.215379,E02002536,122,-1.347271,54.582497,-1.295773,54.610691
1,8449.406849,E02002536,127,-1.214583,54.551181,-1.295773,54.610691
2,8217.024557,E02002536,137,-1.217247,54.552620,-1.295773,54.610691
3,9944.296655,E02002536,836,-1.213812,54.686367,-1.295773,54.610691
4,6707.554555,E02002536,838,-1.216531,54.571757,-1.295773,54.610691
...,...,...,...,...,...,...,...
555153,12599.536095,E02003088,1204,-2.754862,51.433531,-2.815479,51.326784
555154,5650.422116,E02006070,1278,-3.015693,51.082446,-2.977239,51.127113
555155,14527.210489,E02006679,1176,-2.345549,51.360932,-2.136956,51.364067
555156,9895.393238,E02006679,1177,-1.996076,51.352366,-2.136956,51.364067


In [ ]:
euclidean_distance_matrix.to_csv('euclidean_distance_30mile.csv',index=False)

In [ ]:
euclidean_distance_matrix.shape

(555158, 7)

In [ ]:
euclidean_distance_matrix_sub01=euclidean_distance_matrix[0:100000]
euclidean_distance_matrix_sub02=euclidean_distance_matrix[100000:200000]
euclidean_distance_matrix_sub03=euclidean_distance_matrix[200000:300000]
euclidean_distance_matrix_sub04=euclidean_distance_matrix[300000:400000]
euclidean_distance_matrix_sub05=euclidean_distance_matrix[400000:500000]
euclidean_distance_matrix_sub06=euclidean_distance_matrix[500000:555158]

In [ ]:
euclidean_distance_matrix_sub01

,euclidean,origin,dest,site_lon,site_lat,MSOA_lon,MSOA_lat
450000,3396.823841,E02000680,1335,-0.012818,51.429889,-0.061117,51.434470
450001,15263.193208,E02000680,1337,-0.166238,51.554972,-0.061117,51.434470
450002,13846.557898,E02000680,1341,-0.090231,51.557610,-0.061117,51.434470
450003,4683.922776,E02000680,1343,-0.128472,51.435232,-0.061117,51.434470
450004,11551.854101,E02000680,1349,0.104420,51.425747,-0.061117,51.434470
...,...,...,...,...,...,...,...
459995,7108.454715,E02000837,1084,-0.184737,51.480499,-0.092419,51.452955
459996,6602.181578,E02000837,1092,-0.134709,51.506108,-0.092419,51.452955
459997,9986.345486,E02000837,1098,0.025701,51.504128,-0.092419,51.452955
459998,8723.749832,E02000837,1101,-0.207806,51.422142,-0.092419,51.452955


In [ ]:
distance_list = []
origin_id_list = []
destination_id_list = []

for (i1, row1) in euclidean_distance_matrix_sub01.iterrows():
    LatOrigin = str(row1['MSOA_lat'])
    LongOrigin = str(row1['MSOA_lon'])
    origin_id = row1['origin']
    
    LatDestination = str(row1['site_lat'])
    LongDestination = str(row1['site_lon'])
    destination_id = row1['dest']
    
    url = "http://82.11.51.26:5000/route/v1/driving/" + LongOrigin +","+ LatOrigin + ";" + LongDestination +","+ LatDestination + "?steps=false"
    result_distance = requests.get(url).json()['routes'][0]['distance']
    distance_list.append(result_distance)
    origin_id_list.append(origin_id)
    destination_id_list.append(destination_id)

output01 = pd.DataFrame(distance_list, columns = ['Distance in meter'])
output01['origin_id'] = origin_id_list
output01['destination_id'] = destination_id_list
output01

In [ ]:
if len(output01[output01['Distance in meter'].isnull()==True])==0:
    print('There is no empty result.')

There is no empty result.


In [ ]:
from google.colab import files

In [ ]:
output01.to_csv('distance.csv',index=False)

In [ ]:
files.download('distance.csv')

In [ ]:
# These are data generated by Google_distance_cost.ipynb
distance_matrix0 = pd.read_csv('Distance0_100000.csv')
distance_matrix1 = pd.read_csv('Distance100000_200000.csv')
distance_matrix2 = pd.read_csv('Distance200000_300000.csv')
distance_matrix3 = pd.read_csv('Distance300000_400000.csv')
distance_matrix4 = pd.read_csv('Distance400000_1000000.csv')
distance_matrix5 = pd.read_csv('Distance1000000_1674423.csv')

#concat

In [ ]:
distance_matrix = pd.concat([distance_matrix0, distance_matrix1,distance_matrix2,distance_matrix3,distance_matrix4,distance_matrix5], axis=0, ignore_index=True)
distance_matrix

In [ ]:
# check the data
distance_matrix.isna().sum()

In [ ]:
# check the data
distance_matrix[distance_matrix['Distance in meter']==0]

In [ ]:
distance_matrix.to_csv('distance_cost_final_OSRM.csv',index=False)